## Environment Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
%cd rapidsai-csp-utils
!git checkout 22.12-patch
%cd ..
!python rapidsai-csp-utils/colab/env-check.py
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 6.69 MiB/s, done.
Resolving deltas: 100% (191/191), done.
/content/rapidsai-csp-utils
Branch '22.12-patch' set up to track remote branch '22.12-patch' from 'origin'.
Switched to a new branch '22.12-patch'
/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.3 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/env-check.py", line 26, in <module>
    gpu_name = pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0)).decode('UTF-8')
AttributeError: 'str' object has no attribute 'decode'. Did you mean: 'encode'?
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 26, in <module>
    gpu_name =

In [3]:
# Install the Merlin Framework
!pip install -U git+https://github.com/NVIDIA-Merlin/models.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/nvtabular.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/core.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/system.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/dataloader.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/Transformers4Rec.git@release-23.04
!pip install -U xgboost lightfm implicit

  Cloning https://github.com/NVIDIA-Merlin/models.git (to revision release-23.04) to /tmp/pip-req-build-90og30sl
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA-Merlin/models.git /tmp/pip-req-build-90og30sl
  Running command git checkout -b release-23.04 --track origin/release-23.04
  Switched to a new branch 'release-23.04'
  Branch 'release-23.04' set up to track remote branch 'release-23.04' from 'origin'.
  Resolved https://github.com/NVIDIA-Merlin/models.git to commit 56c7d6a4b2ec003472bdd176e972125632d070de
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 6.2 MB/s eta 0:00:00
  Installing buil

## Load Data
Reference
- https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb

In [1]:
import os
import shutil
import numpy as np
from nvtabular import ops
from merlin.schema.tags import Tags
import merlin.io
import merlin.models.tf as mm

import nvtabular as nvt

from os import path

# Get dataframe library - cudf or pandas
from merlin.core.dispatch import get_lib
pd = get_lib()

/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


In [2]:
DATA_DIR = os.environ.get(
    "DATA_DIR", os.path.expanduser("/content/drive/MyDrive/ml-twotower-model/data/")
)

In [8]:
import glob

USE_AUGMENTED_DATASET= True

_TRAIN,_VALID,_TEST = ('train_aug', 'valid_aug', 'test_aug') if USE_AUGMENTED_DATASET else ('train', 'valid', 'test')

TRAIN_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _TRAIN, "*.parquet")))
VALID_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _VALID, "*.parquet")))
TEST_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _TEST, "*.parquet")))
TRAIN_PATHS, VALID_PATHS, TEST_PATHS

(['/content/drive/MyDrive/ml-twotower-model/data/train_aug/part_0.parquet'],
 ['/content/drive/MyDrive/ml-twotower-model/data/valid_aug/part_0.parquet'],
 ['/content/drive/MyDrive/ml-twotower-model/data/test_aug/part_0.parquet'])

In [9]:
train = merlin.io.Dataset(
    TRAIN_PATHS, engine="parquet"
)
valid = merlin.io.Dataset(
    VALID_PATHS, engine="parquet"
)

/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.1

In [10]:
schema = train.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])
schema

,name,tags,dtype,is_list,is_ragged,properties.cat_path,properties.freq_threshold,properties.num_buckets,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.start_index,properties.max_size,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.CATEGORICAL, Tags.ITEM_ID, Tags.ITEM, Ta...","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,100.0,1604.0,0.0,0.0,0,1603,movie_id,NaN,NaN
1,user_id,"(Tags.USER, Tags.CATEGORICAL, Tags.USER_ID, Ta...","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,65.0,752.0,0.0,0.0,0,751,user_id,NaN,NaN
2,gender,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,16.0,3.0,0.0,0.0,0,2,gender,NaN,NaN
3,occupation,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,16.0,22.0,0.0,0.0,0,21,occupation,NaN,NaN
4,genres,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,.//categories/unique.genres.parquet,0.0,None,16.0,20.0,0.0,0.0,0,19,genres,0.0,NaN
5,synopsis,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,.//categories/unique.synopsis.parquet,0.0,None,100.0,1604.0,0.0,0.0,0,1603,synopsis,NaN,NaN
6,title,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,.//categories/unique.title.parquet,0.0,None,100.0,1604.0,0.0,0.0,0,1603,title,NaN,NaN
7,cast,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,.//categories/unique.cast.parquet,0.0,None,181.0,4644.0,0.0,0.0,0,4643,cast,0.0,NaN


In [11]:
#sanity check
train.to_ddf().head()

,movie_id,user_id,gender,occupation,genres,rating,movie_id_raw,synopsis,title,cast
0,121,125,1,6,"[11, 2]",1.0,208,121,121,"[167, 189, 769, 698, 766]"
1,44,234,2,6,"[3, 7, 4, 11]",0.0,183,44,44,"[50, 20, 236, 166, 62, 19, 469]"
2,497,139,1,2,[1],0.0,650,497,497,"[1773, 1783, 1770, 1772, 1774]"
3,1098,8,1,8,[1],0.0,1068,1074,1072,"[3301, 3277]"
4,1082,5,1,1,[1],0.0,1145,1073,1099,"[1032, 3313, 65, 249, 3180, 1261]"


In [12]:
# select only trainable features and exclude the target
train.schema = schema
valid.schema = schema

In [13]:
assert len(schema.select_by_tag(Tags.TARGET).column_names) == 0

## Augment with custom pre-trained embeddings

### Load embeddings

In [19]:
embeddings = pd.read_parquet(os.path.join(DATA_DIR, 'embeddings_index_v2.parquet')).set_index('movie_id')
embeddings.shape

(1603, 2)

In [20]:
embeddings

,title_embeddings,synopsis_embeddings
movie_id,,
279,"[-0.13162778317928314, -0.10936661064624786, -...","[-0.22289082407951355, 0.27888065576553345, 0...."
31,"[0.08139041066169739, -0.16367477178573608, -0...","[-0.11132552474737167, -0.04103025048971176, 0..."
41,"[0.15842819213867188, -0.032023221254348755, -...","[0.07356611639261246, -0.005173327401280403, 0..."
30,"[-0.13973252475261688, -0.19878588616847992, -...","[0.07982735335826874, -0.12655968964099884, 0...."
303,"[-0.18689653277397156, -0.19982194900512695, 0...","[0.06059129536151886, 0.12666839361190796, 0.2..."
...,...,...
1494,"[-0.06935157626867294, -0.3555041551589966, -0...","[-0.04539063572883606, 0.08155060559511185, 0...."
1600,"[-0.10045406222343445, -0.2019217312335968, -0...","[-0.1678621768951416, 0.07765717059373856, 0.0..."
1516,"[-0.26486313343048096, -0.38513806462287903, -...","[-0.23628373444080353, 0.03729664906859398, 0...."


In [21]:
# add embedding for movie_id == 0 for unknown movies

_unknow_movie_embedding = [
    np.random.uniform(-1, 1, size=embeddings['synopsis_embeddings'].iloc[0].shape[0]),
    np.random.uniform(-1, 1, size=embeddings['title_embeddings'].iloc[0].shape[0])
]

In [22]:
embeddings.loc[0] = _unknow_movie_embedding
embeddings.sort_index(inplace=True)

In [23]:
synopsis_embeddings = np.stack(embeddings['synopsis_embeddings'])
title_embeddings = np.stack(embeddings['title_embeddings'])

synopsis_embeddings.shape, title_embeddings.shape

((1604, 768), (1604, 768))

In [24]:
# sanity check
assert (synopsis_embeddings[0] == embeddings.loc[0]['synopsis_embeddings']).all()
assert (title_embeddings[0] == embeddings.loc[0]['title_embeddings']).all()

## Build Model
Reference
- https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb
- https://github.com/NVIDIA-Merlin/models/blob/main/examples/usecases/entertainment-with-pretrained-embeddings.ipynb

### Use Custom Embedding

In [25]:
schema

,name,tags,dtype,is_list,is_ragged,properties.cat_path,properties.freq_threshold,properties.num_buckets,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.start_index,properties.max_size,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.CATEGORICAL, Tags.ITEM_ID, Tags.ITEM, Ta...","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,100.0,1604.0,0.0,0.0,0,1603,movie_id,NaN,NaN
1,user_id,"(Tags.USER, Tags.CATEGORICAL, Tags.USER_ID, Ta...","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,65.0,752.0,0.0,0.0,0,751,user_id,NaN,NaN
2,gender,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,16.0,3.0,0.0,0.0,0,2,gender,NaN,NaN
3,occupation,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,16.0,22.0,0.0,0.0,0,21,occupation,NaN,NaN
4,genres,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,.//categories/unique.genres.parquet,0.0,None,16.0,20.0,0.0,0.0,0,19,genres,0.0,NaN
5,synopsis,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,.//categories/unique.synopsis.parquet,0.0,None,100.0,1604.0,0.0,0.0,0,1603,synopsis,NaN,NaN
6,title,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,.//categories/unique.title.parquet,0.0,None,100.0,1604.0,0.0,0.0,0,1603,title,NaN,NaN
7,cast,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,.//categories/unique.cast.parquet,0.0,None,181.0,4644.0,0.0,0.0,0,4643,cast,0.0,NaN


In [26]:
# swap synopsis embedding
schema['synopsis'].properties['embedding_sizes'] = {
    'cardinality': float(synopsis_embeddings.shape[0]),
    'dimension': float(synopsis_embeddings.shape[1])
}

# swap title embdeddings
schema['title'].properties['embedding_sizes'] = {
    'cardinality': float(title_embeddings.shape[0]),
    'dimension': float(title_embeddings.shape[1])
}

schema

,name,tags,dtype,is_list,is_ragged,properties.cat_path,properties.freq_threshold,properties.num_buckets,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.start_index,properties.max_size,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.CATEGORICAL, Tags.ITEM_ID, Tags.ITEM, Ta...","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,100.0,1604.0,0.0,0.0,0,1603,movie_id,NaN,NaN
1,user_id,"(Tags.USER, Tags.CATEGORICAL, Tags.USER_ID, Ta...","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,65.0,752.0,0.0,0.0,0,751,user_id,NaN,NaN
2,gender,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,16.0,3.0,0.0,0.0,0,2,gender,NaN,NaN
3,occupation,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,None,16.0,22.0,0.0,0.0,0,21,occupation,NaN,NaN
4,genres,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,.//categories/unique.genres.parquet,0.0,None,16.0,20.0,0.0,0.0,0,19,genres,0.0,NaN
5,synopsis,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,.//categories/unique.synopsis.parquet,0.0,None,768.0,1604.0,0.0,0.0,0,1603,synopsis,NaN,NaN
6,title,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,.//categories/unique.title.parquet,0.0,None,768.0,1604.0,0.0,0.0,0,1603,title,NaN,NaN
7,cast,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,.//categories/unique.cast.parquet,0.0,None,181.0,4644.0,0.0,0.0,0,4643,cast,0.0,NaN


In [27]:
# set schema for train and validation set
train.schema = schema
valid.schema = schema

In [28]:
embed_dims = {}
embed_dims["synopsis"] = synopsis_embeddings.shape[1]
embed_dims["title"] = title_embeddings.shape[1]

embeddings_init={
    "synopsis": mm.TensorInitializer(synopsis_embeddings[:1604]),
    "title": mm.TensorInitializer(title_embeddings[:1604]),
}

item_embeddings_block = mm.Embeddings(
    train.schema.select_by_tag(Tags.ITEM),
    infer_embedding_sizes=True,
    embeddings_initializer=embeddings_init,
    trainable={
        'synopsis': False,
        'title': False
        },
    dim=embed_dims,
)

### Model

In [29]:
tower_dim = 64

# create user schema using USER tag
user_schema = schema.select_by_tag(Tags.USER)
# create user (query) tower input block
user_inputs = mm.InputBlockV2(user_schema)
# create user (query) encoder block
query = mm.Encoder(user_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

# create item schema using ITEM tag
item_schema = schema.select_by_tag(Tags.ITEM)
# create item (candidate) tower input block
item_inputs = mm.InputBlockV2(item_schema, categorical=item_embeddings_block)
# create item (candidate) encoder block
candidate = mm.Encoder(item_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

In [30]:
model = mm.TwoTowerModelV2(query, candidate)

In [ ]:
import tensorflow as tf
model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=64, epochs=5)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed i

Epoch 1/5
1250/1250 [==============================] - 40s 27ms/step - loss: 3.6910 - auc: 0.6195 - recall_at_10: 0.4357 - ndcg_at_10: 0.2254 - regularization_loss: 0.0000e+00 - loss_batch: 3.6911 - val_loss: 4.3334 - val_auc: 0.5175 - val_recall_at_10: 0.1858 - val_ndcg_at_10: 0.0864 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.6666
Epoch 2/5
  14/1250 [..............................] - ETA: 21s - loss: 3.6478 - auc: 0.6454 - recall_at_10: 0.4294 - ndcg_at_10: 0.2214 - regularization_loss: 0.0000e+00 - loss_batch: 3.6478

## Evaluate the Model top-k results

In [ ]:
# Top-K evaluation
from merlin.models.utils.dataset import unique_rows_by_features

candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
candidate_features.head()

/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


,movie_id,gender,occupation,genres
0,1,1,1,"[6, 3, 8, 5, 7]"
1,45,1,3,"[2, 5]"
2,357,1,1,"[1, 5]"
3,270,1,2,[1]
4,336,1,15,[4]


In [ ]:
topk = 20
topk_model = model.to_top_k_encoder(candidate_features, k=topk, batch_size=128)

# we can set `metrics` param in the `compile(), if we want
topk_model.compile(run_eagerly=False)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


ValueError: ignored

## TMP

In [ ]:
# model = mm.DLRMModel(
#     train.schema,
#     embedding_dim=64,
#     bottom_block=mm.MLPBlock([128, 64]),
#     top_block=mm.MLPBlock([128, 64, 32]),
#     prediction_tasks=mm.BinaryOutput(
#         train.schema.select_by_tag(Tags.TARGET).column_names[0]
#     ),
# )
# model.compile(optimizer="adam")
# model.fit(train, batch_size=1024)

In [ ]:
metrics = model.evaluate(valid, batch_size=1024, return_dict=True)

metrics